1. socket
2. tcp
3. udp
4. local socket


# 1 socket
---


# 2 socket实现tcp协议
---


# 3 socket实现udp协议
---

# 4 socket实现本地套接字
---
在linux系统中，有一个文件类型是`s`就是咱们这里要介绍的本地套接字(local socket)文件，主要用于信息的传输，其实现方式和tcp的实现方式有些类似，本地套接字的地址族标识是`AF_UNIX`.
> 案例　用local socket 实现双方的信息交互．
- **unix_socket_server.py**,本地套接字的服务端
- **unix_socket_con.py**，本地套接字的连接端


In [ ]:
# unix_socket_server.py
import socket
import os


def create_socket(socket_file, backlog=None):
    socketfd = socket.socket(socket.AF_UNIX, socket.SOCK_STREAM)

    if os.path.exists(socket_file):
        os.remove(socket_file)
    socketfd.bind(socket_file)

    socketfd.listen(backlog)
    return socketfd


def read(conn, buffer_size=4096):
    data = None
    # 如果接收信息发生错误则关闭连接
    try:
        data = conn.recv(buffer_size).decode()
    except OSError as e:
        print('### Not receive msg, close fd:', conn.fileno())
        conn.close()
    return data


def send(conn, msg):
    try:
        conn.send(msg.encode())
    except OSError as e:
        print('### failure send!!')


def main():
    # 本地套接字文件
    socket_file = './socket_file'
    socket_file_name = os.path.basename(socket_file)
    # listen参数
    backlog = 10
    # recv参数
    buffer_size = 1024
    # recv参数
    msg = '***processing***'

    # 创建本地套接字socket实例
    socketfd = create_socket(socket_file, backlog)
    # 等待连接
    conn, _ = socketfd.accept()

    # 持续接收连接端的发送信息并做出响应．
    while True:
        # 读取信息，并打印和发送信息．否则关闭连接并退出循环
        data = read(conn, buffer_size)
        if data:
            print('from %s Receive:%s' % (socket_file_name, data))
            send(conn, msg)
        else:
            break

    socketfd.close()


if __name__ == '__main__':
    main()


In [ ]:
# unix_socket_con.py
import socket


socket_file = './socket_file'
socketfd = socket.socket(socket.AF_UNIX, socket.SOCK_STREAM)
socketfd.connect(socket_file)

while True:
    try:
        msg = input('>>>')
    except (KeyboardInterrupt, EOFError):
        break
    if not msg:
        print('do not input null value!!')
        continue
    socketfd.send(msg.encode())
    data = socketfd.recv(1024).decode()
    if not data:
        break
print('data')

socketfd.close()
